# Model Comparison in The Domain of Brain Tumor Image Classification
For our final project for Spring 2025, CS 4644 - Deep Learning, we analyze and compare the results from three unique models:
1. 3D CNN - Turning 2D images into 3D datapoints to reconstruct a full brain image.
2. ResNet18 - Applying transfer learning by taking a pretrained ResNet-18 model (trained on ImageNet) and adapting it to MRI scans through the fine-tuning of a final fully connected layer.
3. Inception - Applying transfer learning in the same way as ResNet, but for another popular and successful model.

The following code allows the reader to experiment with these 3 models and observe their results.

# Step 0: Get necessary imports and set global variables

In [ ]:
# Basic imports
import sys
import os
import torch
import torch.optim as optim
import torch.nn as nn
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data.Image_Loader import MRIDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import pickle

# Model imports
from src.models.CNN_3D.model import CNN_3D
from src.models.ResNet.model import MyResNet
from src.models.Inception.model import MyInception

# Helper imports
from src.runner import Trainer
from src.optimizer import get_optimizer


from data.data_transforms import (
    get_fundamental_transforms,
    get_fundamental_normalization_transforms,
    get_fundamental_augmentation_transforms,
    get_all_transforms,
)
from utils.utils import compute_mean_and_std
from utils.confusion_matrix import (
    generate_confusion_data,
    generate_confusion_matrix,
    plot_confusion_matrix,
    get_pred_images_for_target,
    generate_and_plot_confusion_matrix,
    generate_and_plot_accuracy_table,
)
from utils.utils import save_trained_model_weights

is_cuda = True
is_cuda = (
    is_cuda and torch.cuda.is_available()
)  # will turn off cuda if the machine doesnt have a GPU

data_path = "./data/"
model_path = "./src/models"
inp_size=(128,128) # determine trhough testing
dataset_mean, dataset_std = compute_mean_and_std(data_path)

In [ ]:
model_resnet = MyResNet()
print(model_resnet)

In [ ]:


# Data preloaded from pickle file
data_path = os.path.abspath(os.path.join("..", "data", "archive", "brain_tumor_mri", "new_dataset", "training_data.pickle"))
with open(data_path, 'rb') as file:
    loaded_data = pickle.load(file)

# Unpacking the data into the images and their corresponding labels
images, labels = zip(*loaded_data)
dataset = MRIDataset(images, labels, transformations, model_type=None)

# Splitting the dataset into training and testing
training_size = int(.8 * len(dataset))
testing_size = len(dataset) - training_size
training_dataset, testing_dataset = random_split(dataset, [training_size, testing_size])

# Two separate loaders for training and testing
train_loader = DataLoader(training_dataset, batch_size=16, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=16, shuffle=True)

# Debug comment out later
for sample_image, sample_label in train_loader:
    print(f"Image shape: {sample_image.shape}")
    print(f"Label: {sample_label}")
    break


# Step 1: Test 3D Convolutional Nerual Network

In [ ]:
model_cnn = CNN_3D(num_classes=3)

optimizer_config = {"optimizer_type": "adam", "lr": 1e-3, "weight_decay": 1e-8} # Tune these
optimizer = get_optimizer(model_cnn, optimizer_config)

trainer = Trainer(
    data_dir=data_path,
    model=model_cnn,
    optimizer=optimizer,
    model_dir=os.path.join(model_path, "CNN_3D"),
    train_data_transforms=get_fundamental_transforms(inp_size),
    val_data_transforms=get_fundamental_transforms(inp_size),
    batch_size=32,
    load_from_disk=False,
    cuda=is_cuda,
)

In [ ]:
%%time
trainer.run_training_loop(num_epochs=50)

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        train_accuracy, validation_accuracy
    )
)

# Step 2: Test ResNet Pretrained Model

In [ ]:
model_resnet = MyResNet()

optimizer_config = {"optimizer_type": "adam", "lr": 1e-3, "weight_decay": 1e-8}
optimizer = get_optimizer(model_resnet, optimizer_config)

trainer = Trainer(
    data_dir=data_path,
    model=model_resnet,
    optimizer=optimizer,
    model_dir=os.path.join(model_path, "ResNet"),
    train_data_transforms=get_fundamental_transforms(inp_size),
    val_data_transforms=get_fundamental_transforms(inp_size),
    batch_size=32,
    load_from_disk=False,
    cuda=is_cuda,
)

In [ ]:
%%time
trainer.run_training_loop(num_epochs=50)

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        train_accuracy, validation_accuracy
    )
)

# Step 3: Test Inception Pretrained Model

In [ ]:
model_inception = MyInception()

optimizer_config = {"optimizer_type": "adam", "lr": 1e-3, "weight_decay": 1e-8}
optimizer = get_optimizer(model_inception, optimizer_config)

trainer = Trainer(
    data_dir=data_path,
    model=model_inception,
    optimizer=optimizer,
    model_dir=os.path.join(model_path, "Inception"),
    train_data_transforms=get_all_transforms(inp_size, [dataset_mean], [dataset_std]),
    val_data_transforms=get_fundamental_normalization_transforms(
        inp_size, [dataset_mean], [dataset_std]
    ),
    batch_size=32,
    load_from_disk=False,
    cuda=is_cuda,
)

In [ ]:
%%time
trainer.run_training_loop(num_epochs=50)

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print(
    "Train Accuracy = {}; Validation Accuracy = {}".format(
        train_accuracy, validation_accuracy
    )
)